In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
import pathlib
import pickle
from pathlib import Path
from skimage.io import imread
import random
import seaborn as sns
sns.set_style('white')
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.utils.plot_funcs import plot_od, plot_train_record

In [4]:
model_path = Path('/mnt/sda1/SMLAT/training_runs/good_models/model_8865_venus.pkl')

In [5]:
with open(model_path, 'rb') as f:
    chromo_model = pickle.load(f)

In [6]:
chromo_model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [8]:
chromo_model.evaluation_params['ground_truth']

[[1,
  1,
  27324.86297607422,
  6616.256332397461,
  25.64394474029541,
  2741.327404975891],
 [2,
  1,
  50220.96221923828,
  6626.470108032227,
  468.40178966522217,
  2499.3234872817993],
 [3,
  1,
  65085.45135498047,
  6612.017288208008,
  140.70481061935425,
  1556.0315251350403],
 [4,
  1,
  66620.42175292969,
  6615.641899108887,
  142.6931619644165,
  1929.4477701187134],
 [5,
  1,
  25949.657135009766,
  6636.897621154785,
  452.75384187698364,
  2268.2154178619385],
 [6,
  1,
  45118.77960205078,
  6725.873908996582,
  195.12498378753662,
  1214.6380841732025],
 [7,
  1,
  58437.99133300781,
  6739.318046569824,
  293.34837198257446,
  1698.6862421035767],
 [8,
  1,
  50595.72784423828,
  6820.064697265625,
  67.0708417892456,
  2436.1692667007446],
 [9,
  1,
  25621.66976928711,
  6856.7060470581055,
  -357.8405976295471,
  1780.7469964027405],
 [10,
  1,
  33593.145751953125,
  6883.284873962402,
  64.59808349609375,
  1684.6014261245728],
 [11,
  1,
  52876.353454589844,

In [7]:
plot_train_record(chromo_model)

In [8]:
from cellbgnet.analyze_eval import recognition, plot_full_img_predictions, assess, assemble_full_img_predictions, limited_matching

#### For simulated data

In [9]:
eval_img = chromo_model.evaluation_params['eval_imgs']

In [10]:
plt.figure()
plt.imshow(eval_img[0])
plt.colorbar()
plt.show()

In [11]:
fov_size = [eval_img.shape[2] * 65, eval_img.shape[1] * 65]

In [12]:
preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=eval_img,
                                             batch_size=16, use_tqdm=False,
                                             nms=True, candidate_threshold=0.3,
                                             nms_threshold=0.7, 
                                             pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                             plot_num=1,
                                             win_size=128,
                                             padding=True,
                                             start_field_pos=[0, 0],
                                             padded_background=chromo_model.evaluation_params['padded_background'])

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]


In [13]:
len(preds_tmp)

7849

##### Getting values of predicted and true x, y, z, i to plot distributions

In [14]:
ground_truth = chromo_model.evaluation_params['ground_truth']

In [15]:
len(ground_truth)

10000

In [16]:
min_int = np.percentile(np.array(ground_truth)[:, -1], 25)

In [17]:
min_int

1645.679846405983

In [18]:
min_int = 1000.0

In [19]:
perf_dict, matches = limited_matching(ground_truth, preds_tmp, min_int=min_int, limited_x=[0, fov_size[0]],
                                     limited_y=[0, fov_size[1]], border=450, print_res=True, tolerance=250,
                                      tolerance_ax=np.inf)

FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 7849
Recall: 0.785
Precision: 1.000
Jaccard: 78.472
RMSE_lat: 52.455
RMSE_ax: 44.178
RMSE_vol: 68.580
Jaccard/RMSE: 1.496
Eff_lat: 43.299
Eff_ax: 69.156
Eff_3d: 56.227
FN: 2152.0 FP: 1.0


In [20]:
matches.shape

(7848, 12)

### Plot distributions of true vs predicted to check for any biases on the full scale

matches is a matrix where rows correspond to a single localization,
columns are as follows. There are 12 columns, I only know what the first 8 are (x and y are in large FOV nm converted)

    1. ground truth x 
    2. ground truth y
    3. ground truth z
    4. ground truth photon counts
    5. predicted x
    6. predicted y
    7. predicted z
    8. predicted photon counts

In [21]:
gt_x, pred_x = matches[:, 0], matches[:, 4]
residual_x = (gt_x - pred_x)
gt_y, pred_y = matches[:, 1], matches[:, 5]
residual_y = (gt_y - pred_y)
gt_z, pred_z = matches[:, 2], matches[:, 6]
residual_z = (gt_z - pred_z)
gt_ph, pred_ph = matches[:, 3], matches[:, 7]
residual_ph = (gt_ph, pred_ph)

In [22]:
plt.figure()
plt.scatter(gt_z, pred_z)
plt.xlabel('Ground truth z')
plt.ylabel('Prediction z')

Text(0, 0.5, 'Prediction z')

In [23]:
plt.figure()
plt.scatter(gt_ph, pred_ph)
plt.xlabel('Ground truth photon counts')
plt.ylabel('Prediction photon counts')

Text(0, 0.5, 'Prediction photon counts')

In [24]:
plt.figure()
plt.hist(gt_z - pred_z, bins=np.arange(-100, 100, 10))
plt.show()

```
List of all matches localizations for further evaluation in format: 
    1. x_gt
    2. y_gt
    3. z_gt
    4. intensity_gt,
    5. x_pred,
    6. y_pred
    7. z_pred
    8. intensity_pred
    9. nms_p
    10.x_sig
    11.y_sig
    12.z_sig
```

In [30]:
sigma_x = np.array([x[7] for x in preds_tmp])
sigma_y = np.array([x[8] for x in preds_tmp])
sigma_z = np.array([x[9] for x in preds_tmp])
sigma_ph = np.array([x[10] for x in preds_tmp])

In [31]:
fig, ax = plt.subplots(nrows=2, ncols=2)
ax[0, 0].hist(sigma_x, bins=25)
ax[0, 0].axvspan(sigma_x.min(), sigma_x.max(), color='green', alpha=0.1)
ax[0, 0].set_xlabel(r'$\sigma_x$ [nm]')
ax[0, 0].set_ylabel('counts')
ax[0, 0].set_title(f"Mean sigma_x is {np.mean(sigma_x):0.3f}, min:{np.min(sigma_x):0.3f}, max:{np.max(sigma_x):0.3f}")

ax[0, 1].hist(sigma_y, bins=25)
ax[0, 1].axvspan(sigma_y.min(), sigma_y.max(), color='green', alpha=0.1)
ax[0, 1].set_xlabel(r'$\sigma_y$ [nm]')
ax[0, 1].set_ylabel('counts')
ax[0, 1].set_title(f"Mean sigma_y is {np.mean(sigma_y):0.3f}, min:{np.min(sigma_y):0.3f}, max:{np.max(sigma_y):0.3f}")


ax[1, 0].hist(sigma_z, bins=25)
ax[1, 0].axvspan(sigma_z.min(), sigma_z.max(), color='green', alpha=0.1)
ax[1, 0].set_xlabel(r'$\sigma_z$ [nm]')
ax[1, 0].set_ylabel('counts')
ax[1, 0].set_title(f"Mean sigma_z is {np.mean(sigma_z):0.3f}, min:{np.min(sigma_z):0.3f}, max:{np.max(sigma_z):0.3f}")

ax[1, 1].hist(sigma_ph, bins=25)
ax[1, 1].axvspan(sigma_ph.min(), sigma_ph.max(), color='green', alpha=0.1)
ax[1, 1].set_xlabel(r'$\sigma_{ph}$ ADU')
ax[1, 1].set_ylabel('counts')
ax[1, 1].set_title(f"Mean sigma_ph is {np.mean(sigma_ph):0.3f}, min:{np.min(sigma_ph):0.3f}, max:{np.max(sigma_ph):0.3f}")

fig.suptitle('Sigmas predicted by the network')
plt.tight_layout()
plt.show()

#### RMSE values distributions

In [32]:
residual_x = np.abs(gt_x - pred_x)
residual_y = np.abs(gt_y - pred_y)
residual_z = np.abs(gt_z - pred_z)
residual_ph = np.abs(gt_ph - pred_ph)

In [33]:
fig, ax = plt.subplots(nrows=2, ncols=2)
ax[0, 0].hist(residual_x, bins=50)
ax[0, 0].axvspan(residual_x.min(), residual_x.max(), color='green', alpha=0.1)
ax[0, 0].set_xlabel(r'$Absoulte distance x $ [nm]')
ax[0, 0].set_ylabel('counts')
ax[0, 0].set_title(f"Mean {np.mean(residual_x):0.3f}, median: {np.median(residual_x):0.3f}, min:{np.min(residual_x):0.3f}, max:{np.max(residual_x):0.3f}")

ax[0, 1].hist(residual_y, bins=50)
ax[0, 1].axvspan(residual_y.min(), residual_y.max(), color='green', alpha=0.1)
ax[0, 1].set_xlabel(r'$Absoulte distance y $ [nm]')
ax[0, 1].set_ylabel('counts')
ax[0, 1].set_title(f"Mean dist y {np.mean(residual_y):0.3f}, median: {np.median(residual_y):0.3f}, min:{np.min(residual_y):0.3f}, max:{np.max(residual_y):0.3f}")

ax[1, 0].hist(residual_z, bins=50)
ax[1, 0].axvspan(residual_z.min(), residual_z.max(), color='green', alpha=0.1)
ax[1, 0].set_xlabel(r'$Absoulte distance z $ [nm]')
ax[1, 0].set_ylabel('counts')
ax[1, 0].set_title(f"Mean dist z {np.mean(residual_z):0.3f}, median: {np.median(residual_z):0.3f}, min:{np.min(residual_z):0.3f}, max:{np.max(residual_z):0.3f}")

ax[1, 1].hist(residual_ph, bins=50)
ax[1, 1].axvspan(residual_ph.min(), residual_ph.max(), color='green', alpha=0.1)
ax[1, 1].set_xlabel(r'$Absoulte difference Photon counts $ ADU')
ax[1, 1].set_ylabel('counts')
ax[1, 1].set_title(f"Mean diff photon counts {np.mean(residual_ph):0.3f}, median: {np.median(residual_ph):0.3f}, min:{np.min(residual_ph):0.3f}, max:{np.max(residual_ph):0.3f}")

fig.suptitle('Absolute distance btn prediction and ground truth')

plt.tight_layout()
plt.show()